In [1]:
base_path =  r'C:\Users\123\Desktop\Python3\HW3'#檔案路徑

## Text Preprocessing

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\123\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from ast import literal_eval
import pandas as pd
import numpy as np
import re
# 自己加的
from collections import defaultdict
from collections import Counter

In [4]:
""" 讀取檔案 """
def read_data(filename):
    data = pd.read_csv(filename, sep='\t')
    data['tags'] = data['tags'].apply(literal_eval)  # str to list
    return data

In [5]:
train = read_data(base_path+'/train.tsv')
validation = read_data(base_path+'/validation.tsv')
test = pd.read_csv(base_path+'/test.tsv', sep='\t')

In [6]:
""" check """
train.head()
train.info
"""
    title
    tag
"""

'\n    title\n    tag\n'

In [7]:
# X_train 儲存所有title，y_train儲存所有tag
X_train, y_train = train['title'].values, train['tags'].values

X_val, y_val = validation['title'].values, validation['tags'].values

X_test = test['title'].values

#### Task 1 (TextPrepare).
#### Implement the function text_prepare following the instructions.text_prepare

In [8]:
# 將以下符號直接替換成空白
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
# 0~9 a~z #+_
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    # lowercase text
    text = text.lower()
    # replace REPLACE_BY_SPACE_RE symbols by space in text
#     temp = re.findall(REPLACE_BY_SPACE_RE,text)
#     if temp :
#         print(temp)
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    
    # delete symbols which are in BAD_SYMBOLS_RE from text
    text = BAD_SYMBOLS_RE.sub('', text)
    # delete stopwords from
    temp = ""
    for word in text.split():
        if word in STOPWORDS:
            pass #如果詞彙是個英文的停用詞的話，就略過不處理。
        else:
            temp += word + " "
    # 把多出的空格拿掉
    temp = temp[:-1]
    
    return temp

In [9]:
def test_text_prepare():
    examples = ["SQL Server - any equivalent of Excel's CHOOSE function?",
                "How to free c++ memory vector<int> * arr?"]
    answers = ["sql server equivalent excels choose function", 
               "free c++ memory vectorint arr"]
    for ex, ans in zip(examples, answers):
        #print(text_prepare(ex))
        if text_prepare(ex) != ans:
            return "Wrong answer for the case: '%s'" % text_prepare(ex)
    return 'Basic tests are passed.'

In [10]:
print(test_text_prepare())

Basic tests are passed.


#### Task 2 (WordsTagsCount)
#### Find 3 most popular tags and 3 most popular words in the train data 

In [11]:
# Dictionary of all tags from train corpus with their counts.
tags_counts = {}
# Dictionary of all words from train corpus with their counts.
words_counts = {}
#words_counts are dictionaries like {'some_word_or_tag': frequency}. 

######################################
######### YOUR CODE HERE #############
for tags in y_train:
    for tag in tags:
        if tag not in tags_counts:
            tags_counts[tag] = 1
        else:
            tags_counts[tag] += 1
#_____________________________________            
#print(X_train)
for titles in X_train:
    titles_after_prepare = text_prepare(titles)
    for word in titles_after_prepare.split():
        if word not in words_counts:
            words_counts[word] = 1
        else:
            words_counts[word] += 1
######################################


In [12]:
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:3]
print(most_common_tags)
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:3]
print(most_common_words)
# print(most_common_words[0][0])

[('javascript', 19078), ('c#', 19077), ('java', 18661)]
[('using', 8278), ('php', 5614), ('java', 5501)]


## Transforming text to a vector
#### Machine Learning algorithms work with numeric data and we cannot use the provided text data "as is". There are many ways to transform text data to numeric vectors. In this task you will try to use two of them.

### Bag of words
#### One of the well-known approaches is a bag-of-words representation. To create this transformation, follow the steps:



1.   Find N most popular words in train corpus and numerate them. Now we have a dictionary of the most popular words.
2.  For each title in the corpora create a zero vector with the dimension equals to N.
3.  For each text in the corpora iterate over words which are in the dictionary and increase by 1 the corresponding coordinate.



In [13]:
# Implement the described encoding in the function my_bag_of_words with the size of the dictionary equals to 5000. 
DICT_SIZE = 5000

####### YOUR CODE HERE #######
word_temp = {}
index_temp = {}
temp = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:DICT_SIZE]
i=0
for element in temp:
    word_temp[element[0]]=i
    index_temp[i]=element[0]
    i=i+1
WORDS_TO_INDEX = word_temp ####### YOUR CODE HERE #######
INDEX_TO_WORDS = index_temp ####### YOUR CODE HERE #######
ALL_WORDS = WORDS_TO_INDEX.keys()

def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    result_vector = np.zeros(dict_size)
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    for word in text.split():
        if word in words_to_index:
            result_vector[words_to_index[word]]+=1
#    print(result_vector)
    return result_vector

In [14]:
def test_my_bag_of_words():
    words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
    examples = ['hi how are you']
    answers = [[1, 1, 0, 1]]
    for ex, ans in zip(examples, answers):
        if (my_bag_of_words(ex, words_to_index, 4) != ans).any():
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

In [15]:
print(test_my_bag_of_words())

Basic tests are passed.


In [16]:
from scipy import sparse as sp_sparse

In [17]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (100000, 5000)
X_val shape  (30000, 5000)
X_test shape  (20000, 5000)


In [18]:
print(type(y_train))

<class 'numpy.ndarray'>


### Task 3 (BagOfWords). 
#### For the 11th row in X_train_mybag find how many non-zero elements it has.

In [19]:
# 不太懂X_train_mybag是什麼資料形式
row = X_train_mybag[10].toarray()[0]
print(X_train_mybag[10])
non_zero_elements_count = 0 ####### YOUR CODE HERE #######
for i in range(0,DICT_SIZE):
    if (row[i]!=0):
        non_zero_elements_count+=1
print("non-zero elements: ",non_zero_elements_count)

  (0, 15)	2.0
  (0, 16)	1.0
  (0, 29)	1.0
  (0, 51)	1.0
  (0, 3095)	1.0
  (0, 4550)	1.0
non-zero elements:  6


### TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result
    
#    tfidf_vectorizer = ######### YOUR CODE HERE #############
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5, token_pattern='(\S+)')
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_val_tfidf = tfidf_vectorizer.transform(X_val)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
    return X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vectorizer.vocabulary_

In [22]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
# tfidf_vocab = words to index 
# rfidf_reversed_vocab = index to words
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

check whether you have c++ or c# in your vocabulary, as they are obviously important tokens in our tags prediction task:

In [23]:
if 'c++' in tfidf_vocab.keys():
  print('c++')
if 'c#' in tfidf_vocab.keys():
  print('c#')

c++
c#


If you can't find it, we need to understand how did it happen that we lost them? It happened during the built-in tokenization of TfidfVectorizer. Luckily, we can influence on this process. Get back to the function above and use '(\S+)' regexp as a token_pattern in the constructor of the vectorizer.

## MultiLabel classifier


*   compare the quality of the bag-of-words and TF-IDF approaches 




In [24]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer

In [25]:
def train_classifier(X_train, y_train, C, regularisation):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
    # Create and fit LogisticRegression wraped into OneVsRestClassifier.

 
    model = OneVsRestClassifier(LogisticRegression(penalty=regularisation, C=C, max_iter=10000)).fit(X_train, y_train)
    return model


mlb_y_train = MultiLabelBinarizer().fit_transform(y_train)
classifier_mybag = train_classifier(X_train_mybag, mlb_y_train, C = 4, regularisation = 'l2')
classifier_tfidf = train_classifier(X_train_tfidf, mlb_y_train, C = 4, regularisation = 'l2')

y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)

y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

#### Evaluation


*   accuracy
*   F1-score macro/micro
* Precision macro/micro



In [26]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

In [27]:
def print_evaluation_scores(y_test, predicted):
    # normalize：默认值为True，返回正确分类的比例；如果为False，返回正确分类的样本数
    print('Accuracy: ', accuracy_score(y_test, predicted, normalize=False))
    print('Accuracy: ', accuracy_score(y_test, predicted))
    print('F1-score macro: ', f1_score(y_test, predicted, average='macro'))
    print('F1-score micro: ', f1_score(y_test, predicted, average='micro'))
    print('F1-score weighted: ', f1_score(y_test, predicted, average='weighted'))
    print('Precision macro: ', average_precision_score(y_test, predicted, average='macro'))
    print('Precision micro: ', average_precision_score(y_test, predicted, average='micro'))
    print('Precision weighted: ', average_precision_score(y_test, predicted, average='weighted'))
    
print('Bag-of-words\n')
mlb_y_val = MultiLabelBinarizer().fit_transform(y_val)
print_evaluation_scores(mlb_y_val, y_val_predicted_labels_mybag)
print('\nTfidf\n')
print_evaluation_scores(mlb_y_val, y_val_predicted_labels_tfidf)

Bag-of-words

Accuracy:  3221
Accuracy:  0.10736666666666667
F1-score macro:  0.21598392084179607
F1-score micro:  0.3135033798056612
F1-score weighted:  0.30259958690895067
Precision macro:  0.10717847317452701
Precision micro:  0.1553448868549377
Precision weighted:  0.20747579302881394

Tfidf

Accuracy:  9895
Accuracy:  0.3298333333333333
F1-score macro:  0.4547756352322309
F1-score micro:  0.6360786856895806
F1-score weighted:  0.6117251205468082
Precision macro:  0.3009512561329642
Precision micro:  0.44480214297739146
Precision weighted:  0.4743105060655193


## Word2Vec

#### ex: mean(word embeddings) --> MLP
#### ex: word embeddings --> LSTM

In [28]:
import gensim
from gensim import models
from gensim.models import Word2Vec, KeyedVectors
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

words_w2v=[]
i=0
for titles in X_train:
    titles_after_prepare = text_prepare(titles)
    temp_list=[]
    for word in titles_after_prepare.split():
        temp_list.append(word)
    words_w2v.append(temp_list)



tokenizer = Tokenizer(num_words=None) # 分词MAX_NB_WORDS
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train) #受num_words影响
word_index = tokenizer.word_index # 词_索引
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences)  #将长度不足 100 的新闻用 0 填充（在前端填充）
labels = to_categorical(np.asarray(mlb_y_train)) #最后将标签处理成 one-hot 向量，比如 6 变成了 [0,0,0,0,0,0,1,0,0,0,0,0,0]，
# print('Shape of data tensor:', data.shape)
# print('Shape of label tensor:', labels.shape)
print(data)

w2v_model = Word2Vec(words_w2v)

Found 26113 unique tokens.
[[    0     0     0 ... 12064     2   134]
 [    0     0     0 ...     3   751    31]
 [    0     0     0 ...   237    87    99]
 ...
 [    0     0     0 ...  3125     5  3644]
 [    0     0     0 ...   373    12   854]
 [    0     0     0 ...     6  2253   269]]


In [29]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [30]:
most_similar(w2v_model, ['using'])

,using,cos
0,via,0.609128
1,problem,0.572076
2,generated,0.568638
3,looping,0.568517
4,export,0.561419
5,retrieved,0.558380
6,scrape,0.557756
7,retrieve,0.553382
8,retrieving,0.550841
9,displaying,0.550452


In [31]:
# Code tried to prepare LSTM model for word generation
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.layers import Dense, Activation

sentence_train=[]
sentence_val=[]
for titles in X_train:
    titles_after_prepare = text_prepare(titles)
    temp_list=[]
    for word in titles_after_prepare.split():
        temp_list.append(word)
    sentence_train.append(temp_list)
    
for titles in X_val:
    titles_after_prepare = text_prepare(titles)
    temp_list_2=[]
    for word in titles_after_prepare.split():
        temp_list_2.append(word)
    sentence_val.append(temp_list_2)
    
for titles in X_val:
    titles_after_prepare = text_prepare(titles)
    temp_list_2=[]
    for word in titles_after_prepare.split():
        temp_list_2.append(word)
    sentence_val.append(temp_list_2)
print (sentence_train[0])
print (sentence_val[0])



w2v_model = Word2Vec(sentence_train, min_count=1,workers=3,window=5)
embedding_layer =Embedding(w2v_model.wv.vectors.shape[0],w2v_model.wv.vectors.shape[1],weights=[w2v_model.wv.vectors])


['draw', 'stacked', 'dotplot', 'r']
['odbc_exec', 'always', 'fail']


In [32]:
#lstm
model_LSTM = Sequential()
model_LSTM.add(embedding_layer)
model_LSTM.add(LSTM(w2v_model.wv.vectors.shape[1]))
model_LSTM.add(Dense(mlb_y_train.shape[1],activation='softmax'))
model_LSTM.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         3149700   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
Total params: 3,240,200
Trainable params: 3,240,200
Non-trainable params: 0
_________________________________________________________________


In [33]:
model_LSTM.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         3149700   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
Total params: 3,240,200
Trainable params: 3,240,200
Non-trainable params: 0
_________________________________________________________________


In [34]:
model_LSTM.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
model_LSTM.fit(x=data, y=mlb_y_train, batch_size=3000, epochs=10)

Epoch 1/10
34/34 [==============================] - 21s 557ms/step - loss: 7.8344 - accuracy: 0.1279
Epoch 2/10
34/34 [==============================] - 20s 580ms/step - loss: 7.5282 - accuracy: 0.1540
Epoch 3/10
34/34 [==============================] - 21s 629ms/step - loss: 7.4733 - accuracy: 0.1516
Epoch 4/10
34/34 [==============================] - 20s 582ms/step - loss: 7.4633 - accuracy: 0.1487
Epoch 5/10
34/34 [==============================] - 20s 585ms/step - loss: 7.4553 - accuracy: 0.1489
Epoch 6/10
34/34 [==============================] - 20s 594ms/step - loss: 7.4453 - accuracy: 0.1474
Epoch 7/10
34/34 [==============================] - 21s 613ms/step - loss: 7.4394 - accuracy: 0.1438
Epoch 8/10
34/34 [==============================] - 21s 621ms/step - loss: 7.4426 - accuracy: 0.1515
Epoch 9/10
34/34 [==============================] - 21s 621ms/step - loss: 7.4363 - accuracy: 0.1630
Epoch 10/10
34/34 [==============================] - 20s 590ms/step - loss: 7.4361 - accura

In [35]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = tokenizer.sequences_to_matrix(sequences, mode='tfidf')
labels = to_categorical(np.asarray(mlb_y_train))

from keras.layers import Dense, Dropout
from keras.models import Sequential

MLP_model = Sequential()
MLP_model.add(Dense(512, input_shape=(len(word_index)+1,), activation='relu'))
MLP_model.add(Dropout(0.2))
MLP_model.add(Dense(labels.shape[1], activation='softmax'))
MLP_model.summary()

Found 26113 unique tokens.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               13370880  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               51300     
Total params: 13,422,180
Trainable params: 13,422,180
Non-trainable params: 0
_________________________________________________________________


In [36]:
# 0.2233
# 0.3062~0.4883

In [37]:
MLP_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
MLP_model.fit(x=data, y=mlb_y_train, batch_size=3000, epochs=10)

Epoch 1/10
34/34 [==============================] - 18s 415ms/step - loss: 6.9216 - accuracy: 0.3472
Epoch 2/10
34/34 [==============================] - 14s 408ms/step - loss: 7.8631 - accuracy: 0.4828
Epoch 3/10
34/34 [==============================] - 14s 419ms/step - loss: 13.7049 - accuracy: 0.4505
Epoch 4/10
34/34 [==============================] - 14s 399ms/step - loss: 22.3702 - accuracy: 0.3747
Epoch 5/10
34/34 [==============================] - 16s 478ms/step - loss: 32.3762 - accuracy: 0.3085
Epoch 6/10
34/34 [==============================] - 15s 439ms/step - loss: 42.4375 - accuracy: 0.2625
Epoch 7/10
34/34 [==============================] - 15s 439ms/step - loss: 51.3588 - accuracy: 0.2264
Epoch 8/10
34/34 [==============================] - 16s 457ms/step - loss: 58.3298 - accuracy: 0.2030
Epoch 9/10
34/34 [==============================] - 17s 492ms/step - loss: 64.0116 - accuracy: 0.1843
Epoch 10/10
34/34 [==============================] - 16s 472ms/step - loss: 69.1175 